In [7]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf
using JLD2

  Activating project at `~/OptimalControl/HW1_S25_Solutions`


# Q3 (30 pts): Log-Domain Interior Point Quadratic Program Solver

Here we are going to use the log-domain interior point method described in Lecture 5 to create a QP solver for the following general problem:

$$\begin{align}
\min_x \quad & \frac{1}{2}x^TQx + q^Tx \\ 
\text{s.t.}\quad &  Ax -b = 0 \\ 
&  Gx - h \geq 0 
\end{align}$$
where the cost function is described by $Q \in \mathbb{R}^{n \times n}$, $q \in \mathbb{R}^n$, an equality constraint is described by $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$, and an inequality constraint is described by $G \in \mathbb{R}^{p \times n}$ and $h \in \mathbb{R}^p$.

We'll first walk you through the steps to reformulate the problem into an interior point log-domain form that we can solve.

## Part (A): KKT Conditions
**TASK**: Introduce Lagrange multipliers $\mu$ for the equality constraint, and $\lambda$ for the inequality constraint and fill in the following for the KKT conditions for the QP. For complementarity use the $\circ$ symbol (i.e. $a \circ b = 0$)

$$\begin{align}
\text{PUT YOUR ANSWER HERE}&= 0 \quad \quad \text{(stationarity)} \\
\text{PUT YOUR ANSWER HERE}&= 0 \quad \quad \text{(primal feasibility)} \\
\text{PUT YOUR ANSWER HERE}&\geq 0 \quad \quad \text{(primal feasibility)} \\
\text{PUT YOUR ANSWER HERE}&\geq 0 \quad \quad \text{(dual feasibility)} \\
\text{PUT YOUR ANSWER HERE}&= 0 \quad \quad \text{(complementarity)} 
\end{align}$$

**SOLUTION**

$$\begin{align}
Qx + q + A^T\mu - G^T\lambda &= 0 \quad \quad \text{(stationarity)} \\
Ax - b &= 0 \quad \quad \text{(primal feasibility)} \\
Gx - h &\geq 0 \quad \quad \text{(primal feasibility)} \\
\lambda &\geq 0 \quad \quad \text{(dual feasibility)} \\
\lambda \circ (Gx - h) &= 0 \quad \quad \text{(complementarity)} 
\end{align}$$

## Part (B): Interior Point Modifications

**TASK**: Modify your KKT conditions by doing the following:
1. Add a slack variable to split the primal feasibility $Gx - h \geq 0$ condition into $Gx - h - s = 0$ and $s \geq 0$
2. Relax the complementarity condition so $\lambda \circ s = 0$ becomes $\lambda \circ s = \rho$ where $\rho$ will be some positive barrier parameter.

Write down the KKT conditions (there should now be six) after you've done the above steps.

$$\begin{align}
\text{PUT YOUR ANSWER HERE}
\end{align}$$

**SOLUTION**

$$\begin{align}
Qx + q + A^T\mu - G^T\lambda = 0 \\
Ax - b = 0 \\
Gx - h - s = 0 \\
s \geq 0 \\
\lambda \geq 0 \\
\lambda \circ s = \rho
\end{align}$$

## Part (C): Log-domain Substitution

**TASK**: Finally do the following:
1. Define a new variable $\sigma$ and define $\lambda = \sqrt\rho e^{-\sigma}$ and $s = \sqrt\rho e^\sigma$. 
2. Replace $\lambda$ and $s$ in your KKT conditions with the new definitions

Three of your KKT conditions from (B) should now be satisfied by construction. Write down the remaining 3 KKT conditions (hint: they should all be $=0$ and the only variables should be x, $\mu$, and $\sigma$).

$$\begin{align}
\text{PUT YOUR ANSWER HERE}
\end{align}$$

**SOLUTION**

$$\begin{align}
Qx + q + A^T\mu - G^T\sqrt\rho e^{-\sigma} = 0 \\
Ax - b = 0 \\
Gx - h - \sqrt\rho e^\sigma = 0 \\
\end{align}$$

## Part (D): Log-domain Interior Point Solver
We can now write our solver! You'll implement two residual functions (matching your residuals in Part A and C), and a function to solve the QP using Newton's method. The solver should work according to the following pseudocode where:
- $\rho$ is the penalty parameter
- kkt_conditions(x, $\mu$, $\lambda$) is the KKT conditions from part A
- ip_kkt_conditions(x, $\mu$, $\sigma$, $\rho$) is the KKT conditions from part C

```
rho = 0.1 (penalty parameter) 
for max_iters
    calculate the Newton step for z using ip_kkt_conditions and ip_kkt_jac
    perform a linesearch to update z (use the same condition as in Q2, with the norm of the ip_kkt_conditions as the merit function)
    if norm(ip_kkt_conditions, Inf) < tol
        rho = rho * 0.1
    end
    if norm(kkt_conditions, Inf) < tol
        exit
    end
end
```

In [24]:
# TODO: read below
# NOTE: DO NOT USE A WHILE LOOP ANYWHERE
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h, xi, μi, σi = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following functions where z = [x; μ; σ]
    
    kkt_res = kkt_conditions(qp, z, ρ)
    ip_res = ip_kkt_conditions(qp, z)
    ip_jac = ip_kkt_jacobian(qp, z)
    x, μ, λ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)

"""

# Helper functions (you can use or not use these)
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end

"""
    kkt_res = kkt_conditions(qp, z, ρ)

Return the KKT conditions from part A as a vector
"""
function kkt_conditions(qp::NamedTuple, z::Vector, ρ::Float64)::Vector
    x, μ, σ = z[qp.xi], z[qp.μi], z[qp.σi]

    λ = sqrt(ρ).*exp.(-σ) # Use element-wise operations to compute ρ

    # SOLUTION
    return [
        qp.Q*x + qp.q + qp.A'*μ - qp.G'*λ
        c_eq(qp, x)
        min.(h_ineq(qp, x), 0)
        min.(λ, 0)
        λ.*h_ineq(qp, x)
    ]

    # TODO compute and return KKT conditions
    error("kkt_conditions not implemented")
    return nothing 
end

"""
    ip_res = ip_kkt_conditions(qp, z)

Return the interior point KKT conditions from part C as a vector
"""
function ip_kkt_conditions(qp::NamedTuple, z::Vector, ρ::Float64)::Vector
    x, μ, σ = z[qp.xi], z[qp.μi], z[qp.σi]

    # SOLUTION
    s, λ = sqrt(ρ).*exp.(σ), sqrt(ρ).*exp.(-σ)
    return [
        qp.Q*x + qp.q + qp.A'*μ - qp.G'*λ
        c_eq(qp, x)
        h_ineq(qp, x) - s
    ]

    # TODO compute and return IP KKT conditions
    error("ip_kkt_conditions not implemented")
    return nothing 
end

"""
    ip_jac = ip_jacobian(qp, z, ρ)

Return the full Newton jacobian of the interior point KKT conditions (part C) with respect to z
"""
function ip_kkt_jac(qp::NamedTuple, z::Vector, ρ::Float64)::Matrix
    x, μ, σ = z[qp.xi], z[qp.μi], z[qp.σi]

    # SOLUTION
    return [
        qp.Q qp.A' qp.G'*diagm(sqrt(ρ).*exp.(-σ))
        qp.A zeros(size(qp.A, 1), size(qp.A, 1) + size(qp.G, 1))
        qp.G zeros(size(qp.G, 1), size(qp.A, 1)) -diagm(sqrt(ρ).*exp.(σ))
    ]

    # TODO: return full Newton jacobian
    error("ip_kkt_jac not implemented")
    return nothing 
end

function logging(qp::NamedTuple, main_iter::Int, z::Vector, ρ::Real, α::Real)
    x, μ, σ = z[qp.xi], z[qp.μi], z[qp.σi]

    # TODO: compute λ
    λ = 0.0

    # TODO: stationarity norm
    stationarity_norm = 0.0 # fill this in 

    # SOLUTIONS
    λ = sqrt(ρ).*exp.(-σ)
    stationarity_norm = norm(qp.Q*x + qp.q + qp.A'*μ - qp.G'*λ)

    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e  %5.0e\n",
          main_iter, stationarity_norm, minimum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(λ,h_ineq(qp,x))), ρ, α)
end

function solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)
    # Init solution vector
    z = zeros(length(qp.q) + length(qp.b) + length(qp.h))

    if verbose
        @printf "iter   |∇Lₓ|      min(h)       |c|       compl     ρ      α\n"
        @printf "----------------------------------------------------------------\n"
    end

    # TODO: implement your solver according to the above pseudocode
    ρ = 0.1
    for main_iter = 1:max_iters 

        # TODO: make sure to save the step length (α) from your linesearch for logging

        # SOLUTION
        ip_res = ip_kkt_conditions(qp, z, ρ)

        ip_jac = ip_kkt_jac(qp, z, ρ)

        Δz = -ip_jac \ ip_res
        α = 1.0
        for ls_iters = 1:10
            if norm(ip_kkt_conditions(qp, z + α*Δz, ρ)) < norm(ip_res)
                break
            end
            α = α/2
        end

        z = z + α*Δz

        if verbose
            logging(qp, main_iter, z, ρ, α) 
        end

        # TODO: convergence criteria based on tol
        # if true
        #     return 0
        # end

        # SOLUTION
        if norm(kkt_conditions(qp, z, ρ), Inf) < tol
            x, μ, λ = z[qp.xi], z[qp.μi], sqrt(ρ).*exp.(-z[qp.σi])
            return x, μ, λ
        elseif norm(ip_kkt_conditions(qp, z, ρ), Inf) < tol
            ρ = ρ * 0.1
        end

    end
    error("qp solver did not converge")
end

solve_qp (generic function with 1 method)

### QP Solver test

In [31]:
# 10 points 
using Test 
@testset "qp solver" begin 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-6)
    
    @load joinpath(@__DIR__, "qp_solutions.jld2") qp_solutions
    @test norm(kkt_conditions(qp, qp_solutions.z, qp_solutions.ρ))<1e-3;
    @test norm(ip_kkt_conditions(qp, qp_solutions.z, qp_solutions.ρ))<1e-3;
    @test norm(ip_kkt_jac(qp, qp_solutions.z, qp_solutions.ρ) - FD.jacobian(dz -> ip_kkt_conditions(qp, dz, qp_solutions.ρ), qp_solutions.z), Inf) < 1e-3
    @test norm(x - qp_solutions.x,Inf)<1e-3;
    @test norm(λ - qp_solutions.λ,Inf)<1e-3;
    @test norm(μ - qp_solutions.μ,Inf)<1e-3;
end

iter   |∇Lₓ|      min(h)       |c|       compl     ρ      α
----------------------------------------------------------------
  1   5.13e+00  -3.51e-01   8.88e-16   6.94e-01  1e-01  1e+00
  2   1.14e+00   6.08e-02   4.44e-16   3.80e-01  1e-01  1e+00
  3   1.16e-01   8.52e-02   1.78e-15   4.52e-01  1e-01  1e+00
  4   5.60e-03   9.01e-02   3.33e-16   4.90e-01  1e-01  1e+00
  5   4.43e-04   9.03e-02   6.66e-16   4.99e-01  1e-01  1e+00
  6   1.23e-06   9.03e-02   3.33e-16   5.00e-01  1e-01  1e+00
  7   7.40e-12   9.03e-02   1.78e-15   5.00e-01  1e-01  1e+00
  8   5.34e-01   3.03e-02   8.88e-16   9.05e-02  1e-02  5e-01
  9   2.25e-02   9.16e-03   8.88e-16   4.93e-02  1e-02  1e+00
 10   1.20e-04   9.28e-03   8.88e-16   5.00e-02  1e-02  1e+00
 11   1.08e-08   9.28e-03   8.88e-16   5.00e-02  1e-02  1e+00
 12   2.83e-01   2.98e-03   8.88e-16   8.59e-03  1e-03  5e-01
 13   9.83e-03   9.35e-04   8.88e-16   4.93e-03  1e-03  1e+00
 14   1.88e-05   9.40e-04   8.88e-16   5.00e-03  1e-03  1e+00
 15   2

Test.DefaultTestSet("qp solver", Any[], 6, false, false, true, 1.737748315300238e9, 1.737748317294878e9, false, "/home/arun/OptimalControl/HW1_S25_Solutions/Q3.ipynb")

# Simulating a Falling Brick with QPs
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \mu \\ \text{ where } M = mI_{2\times 2}, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\mu \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ \Delta t \cdot \begin{bmatrix} \frac{1}{m} J^T \mu_{k+1} - g \\ v_{k+1} \end{bmatrix}$$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\mu_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\mu_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (B): QP formulation for Falling Brick (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + [M (\Delta t \cdot g - v_k)]^Tv_{k+1} \\
    &\text{subject to} && -J(q_k + \Delta t \cdot v_{k+1}) \leq 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown.

**PUT ANSWER HERE:**

## Part (C): Brick Simulation (5 pts)

In [ ]:
function brick_simulation_qp(q, v; mass = 1.0, Δt = 0.01)
    
    # TODO: fill in the QP problem data for a simulation step 
    # fill in Q, q, G, h, but leave A, b the same 
    # this is because there are no equality constraints in this qp 
    
    qp = (
        Q = zeros(2,2), 
        q = zeros(2),
        A = zeros(0,2), # don't edit this
        b = zeros(0),   # don't edit this 
        G = zeros(1,2),
        h = zeros(1)
    )
    
    return qp 
end

In [ ]:
@testset "brick qp" begin 
    
    q = [1,3.0]
    v = [2,-3.0]
    
    qp = brick_simulation_qp(q,v)
    
    # check all the types to make sure they're right
    qp.Q::Matrix{Float64}
    qp.q::Vector{Float64}
    qp.A::Matrix{Float64}
    qp.b::Vector{Float64}
    qp.G::Matrix{Float64}
    qp.h::Vector{Float64}
    
    @test size(qp.Q) == (2,2)
    @test size(qp.q) == (2,)
    @test size(qp.A) == (0,2)
    @test size(qp.b) == (0,)
    @test size(qp.G) == (1,2)
    @test size(qp.h) == (1,)
    
    @test abs(tr(qp.Q) - 2) < 1e-10
    @test norm(qp.q - [-2.0, 3.0981]) < 1e-10 
    @test norm(qp.G - [0 -.01]) < 1e-10 
    @test abs(qp.h[1] -3) < 1e-10
    
end

In [ ]:
include(joinpath(@__DIR__, "animate_brick.jl"))
let 
    
    dt = 0.01 
    T = 3.0 
    
    t_vec = 0:dt:T
    N = length(t_vec)
    
    qs = [zeros(2) for i = 1:N]
    vs = [zeros(2) for i = 1:N]
    
    qs[1] = [0, 1.0]
    vs[1] = [1, 4.5]
    
    # TODO: simulate the brick by forming and solving a qp 
    # at each timestep. Your QP should solve for vs[k+1], and
    # you should use this to update qs[k+1]

    
    xs = [q[1] for q in qs]
    ys = [q[2] for q in qs]
    
    @show @test abs(maximum(ys)-2)<1e-1
    @show @test minimum(ys) > -1e-2
    @show @test abs(xs[end] - 3) < 1e-2
    
    xdot = diff(xs)/dt
    @show @test maximum(xdot) < 1.0001
    @show @test minimum(xdot) > 0.9999
    @show @test ys[110] > 1e-2
    @show @test abs(ys[111]) < 1e-2
    @show @test abs(ys[112]) < 1e-2
    
    display(plot(xs, ys, ylabel = "y (m)", xlabel = "x (m)"))
    
    animate_brick(qs)
    
    
    
end

# Part D (5 pts): Solve a QP

Use your QP solver to solve the following optimization problem:


$$
\begin{align} 
\min_{y\in\mathbb{R}^2,a\in\mathbb{R},b\in\mathbb{R}} \quad & \frac{1}{2}y^T \begin{bmatrix} 1 & .3 \\ .3 & 1 \end{bmatrix} y + a^2 + 2b^2  + \begin{bmatrix} -2 & 3.4 \end{bmatrix} y + 2a + 4b \\ 
\text{st} \quad & a + b = 1 \\ 
& \begin{bmatrix}-1 & 2.3 \end{bmatrix} y + a - 2b =3 \\
& -0.5 \leq y \leq 1 \\ 
& -1 \leq a \leq 1 \\ 
& -1 \leq b \leq 1
\end{align}
$$

You should be able to put this into our standard QP form that we used above, and solve.

In [ ]:
@testset "part D" begin

    y = randn(2)
    a = randn()
    b = randn()
    
    
    @test norm(y - [-0.080823; 0.834424]) < 1e-3 
    @test abs(a - 1) < 1e-3 
    @test abs(b) < 1e-3 
end

## Part E (5 pts): One sentence short answer

1. For our Augmented Lagrangian solver, if our initial guess for $x$ is feasible (meaning it satisfies the constraints), will it stay feasible through each iteration? 

**put ONE SENTENCE answer here**

2. Does the Augmented Lagrangian function for this problem always have continuous first derivatives?

**put ONE SENTENCE answer here** 

3. Is the QP in part D always convex?

**put ONE SENTENCE answer here**